The goal is to do topic modeling over all the mails. In other words, we have to find recurrent topic or themes that may appear in the conversations.
They are several way to analyse the mails content, starting by these two "naive" ways:
- put all the extrated mails in only one document
- put each extracted mail in a separate document

But both of these ways have major drawbacks:
- doing topic modelling on a single document would show the most frequent words, so the result should be the same as if we wanted to make a word cloud
- a lot of mail are very small, a few words sometimes, so doing topic analysis here would be meaningless

So we have to find a compromise: make multiple documents, each of them long enough to be analysed.
One option would be create the entire conversations with the mail history, so we can extract main topic from each conversation. While it makes sense, it's actually pretty time-consuming to obtain the conversations.

What we will do is to create a document that contains the "sent mails box" for each person. It doesn't follow a conversation, so our results won't be the most coherent we could get. But the purpose here is to show the basics of topic modelling.

In [36]:
import pandas as pd
import gensim
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re # regular expressions

In [18]:
emails = pd.read_csv("hillary-clinton-emails/Emails.csv")

In [19]:
# Drop columns that won't be used
emails = emails.drop(['DocNumber', 'MetadataPdfLink','DocNumber', 'ExtractedDocNumber', 'MetadataCaseNumber'], axis=1)

In [55]:
sampleEmail = emails.loc[1].ExtractedBodyText

In [66]:
emails.ExtractedBodyText

0                                                     NaN
1       B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: La...
2                                                     Thx
3                                                     NaN
4       H <hrod17@clintonemail.com>\r\nFriday, March 1...
5       Pis print.\r\n-•-...-^\r\nH < hrod17@clintoner...
6                                                     NaN
7       H <hrod17@clintonemail.corn>\r\nFriday, March ...
8                                                     FYI
9       B6\r\nWednesday, September 12, 2012 6:16 PM\r\...
10                                       Fyi\r\nB6\r\n— —
11      B6\r\nWednesday, September 12, 2012 6:16 PM\r\...
12                                                    Fyi
13      Anne-Marie Slaughter\r\nSunday, March 13, 2011...
14      _ .....\r\nFrom Randolph, Lawrence M\r\nSent: ...
15      I asked to attend your svtc today with Embassy...
16                   Hope. See picture below Kamala sent.
17            

In [65]:
for text in emails.ExtractedBodyText:
    tokens_list = tokenizer(text)

AttributeError: 'float' object has no attribute 'replace'

In [42]:
# Testing the cleaning function
sample_tokens = tokenizer(sampleEmail)
sample_tokens

['B6',
 'Thursday',
 'March',
 '3',
 '2011',
 '9',
 '45',
 'PM',
 'H',
 'Latest',
 'How',
 'Syria',
 'is',
 'aiding',
 'Qaddafi',
 'and',
 'more',
 'Sid',
 'hrc',
 'memo',
 'syria',
 'aiding',
 'libya',
 '030311',
 'docx',
 'hrc',
 'memo',
 'syria',
 'aiding',
 'libya',
 '030311',
 'docx',
 'March',
 '3',
 '2011',
 'For',
 'Hillary']

In [72]:
emailsBySend = emails.groupby(['SenderPersonId'])['ExtractedBodyText']
df = list(emailsBySend)
df = pd.DataFrame(df)

In [39]:
senderIDs = []
for senderID in emails['SenderPersonId']:
#    print(senderID)

SyntaxError: unexpected EOF while parsing (<ipython-input-39-6c06c0c33ed8>, line 3)

In [32]:
emailsBySend

In [20]:
bodyContent = pd.DataFrame(emails.ExtractedBodyText.dropna())

In [21]:
bodyContent

,ExtractedBodyText
1,"B6\r\nThursday, March 3, 2011 9:45 PM\r\nH: La..."
2,Thx
4,"H <hrod17@clintonemail.com>\r\nFriday, March 1..."
5,Pis print.\r\n-•-...-^\r\nH < hrod17@clintoner...
7,"H <hrod17@clintonemail.corn>\r\nFriday, March ..."
8,FYI
9,"B6\r\nWednesday, September 12, 2012 6:16 PM\r\..."
10,Fyi\r\nB6\r\n— —
11,"B6\r\nWednesday, September 12, 2012 6:16 PM\r\..."
12,Fyi


This part contains used the source code that was used above during the homework, just for your curiosity.

In [28]:
# Words to be removed because they "falsify" the results
ignored_words = ['re', 'fw', 'pm']

In [60]:
def tokenizer(text):
    cleanedText = text.replace('\n', ' ')
    cleanedText = cleanedText.replace('\r', '')
    # Separate each word
    tokens = RegexpTokenizer(r'\w+').tokenize(cleanedText)
    
    return tokens